In [1]:
from dataprocess.kdd_cup99 import KDD_CUP_99_DataLoader
from net.CNN_KDD import CNN_KDD
from net.multCNN import multCNN

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter

In [3]:
import os 
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]='0'

device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [4]:
dataset = KDD_CUP_99_DataLoader('E:/DataSets/kddcup.data', 256)
discrete_column_idx = dataset.data.discrete_column
discrete_column_idx.remove(41)
continuous_column_idx = [i for i in range(41) if i not in discrete_column_idx]
input_channel_kind = len(discrete_column_idx)
input_channel_num = len(continuous_column_idx)

In [5]:
# net = CNN_KDD(41,23,3).to(device)
net = multCNN(input_channel_num, input_channel_kind, 23).to(device)
epoch = 100
criterion = nn.CrossEntropyLoss().to(device)
optimizer = optim.SGD(net.parameters(), lr=1e-2)

In [7]:
writer1 = SummaryWriter('log/exp')

In [11]:
# ckpt = torch.load('/Transfer-learning-IDS/history/CNN_KDD/checkpoint/ckpt_best_99.pth')
# epoch_start = ckpt["epoch"]
# net.load_state_dict(ckpt["net"])
# optimizer.load_state_dict(ckpt["optimizer"])

In [9]:
for t in range(0, epoch):
    running_loss = 0
    for step, (x, y) in enumerate(dataset):
        
        x_continuous, x_discrete =x[:, continuous_column_idx], x[:, discrete_column_idx]
        x_continuous = x_continuous.to(device)
        x_discrete = x_discrete.to(device)
        y = F.one_hot(y.long(), num_classes=23).float()
        y = y.to(device)
        
        # Zero gradients, perform a backward pass, and update the weights.
        optimizer.zero_grad()
        y_pred = net((x_continuous, x_discrete))
        loss = criterion(y_pred, y)
        loss.backward()
        optimizer.step()
        
        # visualize loss
        running_loss += loss.item()
    # ...log the running loss
    writer1.add_scalar('training loss', running_loss, t)
    if t % 10 == 9:
        checkpoint = {"net": net.state_dict(), 'optimizer':optimizer.state_dict(), "epoch": t}
        if not os.path.isdir("/Transfer-learning-IDS/history/multCNN/checkpoint"):
            os.makedirs("/Transfer-learning-IDS/history/multCNN/checkpoint")
        torch.save(checkpoint, '/Transfer-learning-IDS/history/multCNN/checkpoint/ckpt_best_%s.pth' %(str(t)))
writer1.close()

KeyboardInterrupt: 